In [ ]:
%pip install transformers

Defaulting to user installation because normal site-packages is not writeable
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
import ipaddress
import time
import datetime
import re

import gc
import random

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# EDA

In [ ]:
df = pd.read_csv('part_10.csv.gz')

In [ ]:
df.dropna(axis= 0, how='all', inplace= True)
df.drop(columns= ['EVENT_ID', 'RESPONSE_CODE'], inplace= True)

In [ ]:
df.isna().sum()

CLIENT_IP                    0
CLIENT_USERAGENT           321
REQUEST_SIZE                44
MATCHED_VARIABLE_SRC       108
MATCHED_VARIABLE_NAME     4671
MATCHED_VARIABLE_VALUE     155
dtype: int64

In [ ]:
df

,CLIENT_IP,CLIENT_USERAGENT,REQUEST_SIZE,MATCHED_VARIABLE_SRC,MATCHED_VARIABLE_NAME,MATCHED_VARIABLE_VALUE
0,188.138.92.55,NaN,166,REQUEST_URI,NaN,//tmp/20160925122692indo.php.vob
1,93.158.215.131,Mozilla/5.0 (Windows NT 6.3; WOW64; rv:45.0) G...,431,REQUEST_GET_ARGS,url,http://www.galitsios.gr/?option=com_k2
2,176.123.240.64,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...,395,REQUEST_PATH,NaN,"/opinion/page1_3.php\"" and \""\""x\""\""=\""\""x\"""
3,73.78.81.63,Mozilla/5.0 (Windows NT 6.3; WOW64; rv:45.0) G...,387,REQUEST_GET_ARGS,url,http://thereviewgroup.org/boss-capital/
4,66.102.9.44,Mozilla/5.0 (Linux; Android 4.4.2; GT-N7100 Bu...,1733,REQUEST_GET_ARGS,REQUEST_GET_ARGS._,1516288084140
...,...,...,...,...,...,...
57840,217.175.140.69,MICROSOFT_DEVICE_METADATA_RETRIEVAL_CLIENT,2402,REQUEST_XML,REQUEST_XML./s:Envelope/s:Body/DeviceMetadataB...,DOID:USBSTOR\\Imation_USB_Flash_Drive_2
57841,109.206.151.20,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,922,REQUEST_GET_ARGS,REQUEST_GET_ARGS._,1516440395935
57842,89.23.32.19,Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:57...,1453,REQUEST_COOKIES,REQUEST_COOKIES.User-cn,Николай Харченко
57843,194.228.20.50,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,1550,REQUEST_POST_ARGS,REQUEST_POST_ARGS.s_Unids,688D71625B88803CC125818E00471A8D;9915FD3520134...


## Основная мысль следующая:
Разделить на хорошие и плохие запросы, далее взять юзер агента и значение которое было передано в запросе нормализовать и скормить все трансформеру, так же можно воспользоваться fast-text и классифицировать на его эмбедингах, но bert выглядит понадежнее.

In [ ]:
def clean_n(x):
  if isinstance(x, type(np.nan)):
    return 0
  if re.findall(r'\b[0-9]+\b', x):
    return int(x)
  else:
    return -1

In [ ]:
def tag_ip(x): #проверка валидности ip
  if x.count('.') == 3:
    try:
      ipaddress.IPv4Address(x)
      return 0
    except ipaddress.AddressValueError:
      return 1
  else:
    try:
      ipaddress.IPv6Address(x)
      return 0
    except ipaddress.AddressValueError:
      return 1

In [ ]:
def bad_req_tagger(x, useragent=True):
  if isinstance(x, str):
    for c in code_ex:
      if x.lower().find(c) != -1 and x.lower().find('mozilla') == -1 and x.lower().find('opera') == -1:
        return 1
    if x[0] in stop_s:
      return 1
    return 0
  else:
    return 1

## Список плохих запрсов:
Чтобы разметить данные надо сводить их постепенно к общему знаменателю:
Смотрим на данные находим например что тыкают ковычку - пишем в лист с примерами плохих запросов, в юзер агенте остались какие то неизмветсные названия - загуглить что это или спросить у GPT, нашли base64 строчку - пытаемся расшифровать посмотреть что внутри(кстати не получилось), в общем в ручную по крупице собираем target
В основном для нас важны стобцы с ip, user-agent и value

In [ ]:
stop_s = ['*', '@', '"', '|', '{', '(', "'", '$']
code_ex = ['print', 'select', 'avd', 'waitfor', '-1 or', 'ls', '-la', 'factor', 'ltx71', 'bot', 'b-o-t', 'b o t', 'xml', '${',
           'script', 'php', '==', 'int', 'varchar', 'sql', 'curl', 'benchhttp', 'zmeu', 'alert',
           'python', 'requests', 'arachni', 'xenu', 'nessus', 'scrapy', 'wget', 'urlgrabber',
           'jigsaw', 'nsplayer', 'crawler', 'openvas', 'detector', 'or']

In [ ]:
df.REQUEST_SIZE = df.REQUEST_SIZE.apply(lambda x: clean_n(x))#инсерт рядом с колонкой для наглядности
df.insert(1, 'IP_Tag',df.CLIENT_IP.apply(lambda x: tag_ip(x)))
df.insert(3, 'USERAGENT_Tag',df.CLIENT_USERAGENT.apply(lambda x: bad_req_tagger(x)))
# df.insert(8, 'VARIABLE_NAME', df.MATCHED_VARIABLE_NAME.apply(lambda x: x.split('.')[1] if isinstance(x, str) and len(x.split('.')) > 1 else x))
df.insert(8, 'VARIABLE_VALUE', df.MATCHED_VARIABLE_VALUE.apply(lambda x: bad_req_tagger(x, useragent= False)))

In [ ]:
df

,CLIENT_IP,IP_Tag,CLIENT_USERAGENT,USERAGENT_Tag,REQUEST_SIZE,MATCHED_VARIABLE_SRC,MATCHED_VARIABLE_NAME,MATCHED_VARIABLE_VALUE,VARIABLE_VALUE
0,188.138.92.55,0,NaN,1,166,REQUEST_URI,NaN,//tmp/20160925122692indo.php.vob,1
1,93.158.215.131,0,Mozilla/5.0 (Windows NT 6.3; WOW64; rv:45.0) G...,0,431,REQUEST_GET_ARGS,url,http://www.galitsios.gr/?option=com_k2,0
2,176.123.240.64,0,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...,0,395,REQUEST_PATH,NaN,"/opinion/page1_3.php\"" and \""\""x\""\""=\""\""x\""",1
3,73.78.81.63,0,Mozilla/5.0 (Windows NT 6.3; WOW64; rv:45.0) G...,0,387,REQUEST_GET_ARGS,url,http://thereviewgroup.org/boss-capital/,1
4,66.102.9.44,0,Mozilla/5.0 (Linux; Android 4.4.2; GT-N7100 Bu...,0,1733,REQUEST_GET_ARGS,REQUEST_GET_ARGS._,1516288084140,0
...,...,...,...,...,...,...,...,...,...
57840,217.175.140.69,0,MICROSOFT_DEVICE_METADATA_RETRIEVAL_CLIENT,0,2402,REQUEST_XML,REQUEST_XML./s:Envelope/s:Body/DeviceMetadataB...,DOID:USBSTOR\\Imation_USB_Flash_Drive_2,1
57841,109.206.151.20,0,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,0,922,REQUEST_GET_ARGS,REQUEST_GET_ARGS._,1516440395935,0
57842,89.23.32.19,0,Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:57...,0,1453,REQUEST_COOKIES,REQUEST_COOKIES.User-cn,Николай Харченко,0
57843,194.228.20.50,0,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,0,1550,REQUEST_POST_ARGS,REQUEST_POST_ARGS.s_Unids,688D71625B88803CC125818E00471A8D;9915FD3520134...,0


In [ ]:
df['labels'] = [(a or b or c) for a, b, c in zip(df.IP_Tag.values, df.USERAGENT_Tag.values, df.VARIABLE_VALUE.values)]

In [ ]:
df['labels'].value_counts()

0    41639
1    16199
Name: labels, dtype: int64

# TRAIN

## Нормализация
Основная идея в том чтобы убрать лишнее и не задеть нужное, например для нас важны * @ " { } так как там может быть исполняемый код, а _ или \\ или , для нас не так важны в тексте

In [ ]:
def cleaning(x):
  if isinstance(x, str):
    x = re.sub('[0-9]+', '', x)
    x = re.sub(r'[|;|:|.|,|/|_|\\]', ' ', x)
    x = re.sub(r'[-]', ' ', x)
    x = re.sub(' +', ' ', x).lower()
    x = x[:500]
  return x

In [ ]:
df['useragent+value'] = df.CLIENT_USERAGENT.apply(lambda x: cleaning(x)) + ' ' + df.MATCHED_VARIABLE_VALUE.apply(lambda x: cleaning(x))

In [ ]:
clean_df = pd.DataFrame(df[['useragent+value', 'labels']])
clean_df.dropna(axis=0, inplace= True)
clean_df.reset_index(inplace= True, drop=True)
clean_df

,useragent+value,labels
0,mozilla (windows nt wow rv ) gecko firefox ht...,0
1,mozilla (windows nt rv ) gecko firefox opini...,1
2,mozilla (windows nt wow rv ) gecko firefox ht...,1
3,mozilla (linux android gt n build koth) applew...,0
4,mozilla (linux android lg h build mrak) applew...,0
...,...,...
57384,microsoft device metadata retrieval client doi...,1
57385,mozilla (windows nt wow) applewebkit (khtml li...,0
57386,mozilla (windows nt win x rv ) gecko firefox ...,0
57387,mozilla (windows nt wow trident rv ) like geck...,0


In [ ]:
clean_df['useragent+value'].str.len().max()

668

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(clean_df['useragent+value'], clean_df.labels, test_size=0.20, random_state=42, shuffle=True, stratify=clean_df.labels)

In [ ]:
pd.DataFrame([X_test,y_test]).T.to_csv('test.csv', index= False)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
max_len = 0
for sent in clean_df['useragent+value'].values:

    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    max_len = max(max_len, len(input_ids))

#найдем максимальную длину текста в корпусе
print('Max sentence length: ', max_len)

Max sentence length:  474


In [ ]:
input_ids = []
attention_masks = []

for sent in X_train.values:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = max_len,
                        truncation=True,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   ) #токенизируем

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(y_train.values)

print('Original: ', X_train.iloc[0])
print('Token IDs:', input_ids[0])

/home/jupyter/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  mozilla (windows nt wow) applewebkit (khtml like gecko) chrome safari  post ' 'xmlhttprequest ' 'aeddbbfabefdacbcebabffeab
Token IDs: tensor([  101,  9587,  5831,  4571,  1006,  3645, 23961, 10166,  1007,  6207,
         8545,  2497, 23615,  1006,  1047, 11039, 19968,  2066, 16216, 19665,
         1007, 18546, 23591,  2695,  1005,  1005, 20950, 11039, 25856,  2890,
        15500,  1005,  1005, 29347, 14141, 10322,  7011,  4783,  2546,  2850,
        27421,  3401,  3676, 29292,  7959,  7875,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     

In [ ]:
dataset = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.8 * len(dataset))
val_size = len(dataset)  - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

36,728 training samples
9,183 validation samples


In [ ]:
z_t, z_v= [],[]
t_t, t_v =[],[]
for t,v in zip(train_dataset, val_dataset):
  if t[2].item() == 1:
    t_t.append(1)
  if v[2].item() == 1:
    t_v.append(1)

In [ ]:
len(t_t), len(t_v) #проверим распределение классов

(2487, 2439)

In [ ]:
batch_size = 32

train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

if device == "cuda:0":
  model = model.cuda()
else:
  model = model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, #возьмем немного меньше чем по дефолту
                  eps = 1e-8
                )

/home/jupyter/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
epochs = 4 #можно взять и больше но не вижу смысла

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []

total_t0 = time.time()

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)
        loss = output.loss
        total_train_loss += loss.item()

        loss.backward()#распространяем обратно
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)#не взрываем градиент
        optimizer.step()#обновляем
        scheduler.step()#обновляем

    #AVG los
    avg_train_loss = total_train_loss / len(train_dataloader)

    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    # ========================================
    #               Validation
    # ========================================
    print("")
    print("Running Validation...")
    t0 = time.time()
    model.eval()
    total_eval_accuracy = 0
    best_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device).
        with torch.no_grad():
            output= model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = output.loss
        total_eval_loss += loss.item()
        #если используете GPU ставьте CPU ниже
        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)
    if avg_val_accuracy > best_eval_accuracy:
        torch.save(model, 'bert_model')
        best_eval_accuracy = avg_val_accuracy

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.16
  Training epcoh took: 0:16:52

Running Validation...
  Accuracy: 0.96

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.09
  Training epcoh took: 0:16:47

Running Validation...
  Accuracy: 0.97

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.07
  Training epcoh took: 0:16:47

Running Validation...
  Accuracy: 0.97

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.05
  Training epcoh took: 0:16:47

Running Validation...
  Accuracy: 0.97

Training complete!
Total training took 1:12:58 (h:mm:ss)


### Можно было еще добавить REQUEST_SIZE но не думаю что это бы дало какой то критический прирост


# TEST

In [ ]:
model = torch.load('bert_model')

In [ ]:
df_test = pd.read_csv('test.csv')
df_test

,useragent+value,labels
0,mozilla (windows nt ) applewebkit (khtml like ...,0.0
1,mozilla (compatible msie windows nt sv net clr...,1.0
2,mozilla (windows nt win x) applewebkit (khtml ...,0.0
3,mozilla (windows nt wow) applewebkit (khtml li...,0.0
4,mozilla (windows nt wow) applewebkit (khtml li...,0.0
...,...,...
11475,mozilla (compatible yandexbot +http yandex com...,1.0
11476,mozilla (linux android sm jf build lmyx) apple...,0.0
11477,msdw dmsch,0.0
11478,mozilla (linux u android en us redmi build mmb...,0.0


In [ ]:
test_text = df_test['useragent+value'].values

In [ ]:
test_input_ids = []
test_attention_masks = []
for sent in test_text:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = max_len,
                        truncation=True,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    test_input_ids.append(encoded_dict['input_ids'])
    test_attention_masks.append(encoded_dict['attention_mask'])
test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
test_dataset = TensorDataset(test_input_ids, test_attention_masks)
test_dataloader = DataLoader(
            test_dataset,
            sampler = SequentialSampler(test_dataset),
            batch_size = batch_size
        )

In [ ]:
predictions = []
for batch in test_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        with torch.no_grad():
            output= model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask)
            logits = output.logits
            logits = logits.detach().cpu().numpy()
            pred_flat = np.argmax(logits, axis=1).flatten()

            predictions.extend(list(pred_flat))

In [ ]:
df_test.fillna(0, inplace=True)

In [ ]:
df_test.isna().sum()

useragent+value    0
labels             0
dtype: int64

In [ ]:
print(classification_report(df_test.labels.values, predictions))

              precision    recall  f1-score   support

         0.0       0.98      0.97      0.98      8330
         1.0       0.93      0.95      0.94      3150

    accuracy                           0.97     11480
   macro avg       0.96      0.96      0.96     11480
weighted avg       0.97      0.97      0.97     11480

